In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import scipy.io
from torchsummary import summary

In [2]:
BATCH_SIZE=10 
EPOCHS=100 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [3]:
class ConvNet_BLT(nn.Module):
    def __init__(self,dropout_p = 0.9):
        super().__init__()
        self.dropout_p = dropout_p
        for i in range(1,11):
            setattr(self,'conv'+str(i)+'1',nn.Conv2d(1, 32, 3,stride = 1))
            setattr(self,'conv'+str(i)+'2',nn.Conv2d(32, 64, 3,stride = 1))
            setattr(self,'deconv'+str(i)+'2',nn.ConvTranspose2d(64,32,3))
            setattr(self,'deconv'+str(i)+'1',nn.ConvTranspose2d(32,1,3))
            setattr(self,'conv'+str(i)+'3',nn.Conv2d(64,1 , 3))
            
    def forward(self,x):
        in_size = x.size(0) 
        out_1 = x.clone() 
        out_f = []
        for i in range(1,11):
            out = eval('self.conv'+str(i)+'1')(x) 
            out = F.relu(out) 
            out_ll = out.clone()
            out_ll = eval('self.deconv'+str(i)+'1')(out_ll)
            out_ll = F.relu(out_ll)
            out_temp = out.clone() 
            out_temp = eval('self.conv'+str(i)+'2')(out_temp)
            out_temp = F.relu(out_temp)
            out_2 = out_temp.clone()
            out_2l = eval('self.deconv'+str(i)+'2')(out_temp) 
            out_2l = F.relu(out_2l)
            
            
            out = eval('self.conv'+str(i)+'3')(out_2)
            out_t = F.adaptive_avg_pool2d(out,(1,1)) 
                
            out_1r = eval('self.deconv'+str(i)+'1')(out_2l.clone())
            out_1r = F.relu(out_1r)
            
            for s in range(3):
                out = out_ll + out_1 + out_1r
                out = eval('self.conv'+str(i)+'1')(out) # 
                out = F.relu(out) 
                out_ll = out.clone()
                out_ll = eval('self.deconv'+str(i)+'1')(out_ll)
                out_ll = F.relu(out_ll)

                out_temp = out + out_2l
                out_temp = eval('self.conv'+str(i)+'2')(out_temp)
                out_temp = F.relu(out_temp)
                
                out_2 = out_temp.clone() 
                out_2l = eval('self.deconv'+str(i)+'2')(out_temp) 
                out_2l = F.relu(out_2l)
                
                out_1r = eval('self.deconv'+str(i)+'1')(out_2l.clone())
                out_1r = F.relu(out_1r)
                
                out = eval('self.conv'+str(i)+'3')(out_2)
                out_t += F.adaptive_avg_pool2d(out,(1,1))

                out_00 = out_t.view(in_size, -1).clone()
            out_f.append(out_00)
            

        output = torch.cat(out_f,dim=1)
        return output

print(summary(ConvNet_BLT().to(DEVICE), (1, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             320
   ConvTranspose2d-2            [-1, 1, 32, 32]             289
            Conv2d-3           [-1, 64, 28, 28]          18,496
   ConvTranspose2d-4           [-1, 32, 30, 30]          18,464
            Conv2d-5            [-1, 1, 26, 26]             577
   ConvTranspose2d-6            [-1, 1, 32, 32]             289
            Conv2d-7           [-1, 32, 30, 30]             320
   ConvTranspose2d-8            [-1, 1, 32, 32]             289
            Conv2d-9           [-1, 64, 28, 28]          18,496
  ConvTranspose2d-10           [-1, 32, 30, 30]          18,464
  ConvTranspose2d-11            [-1, 1, 32, 32]             289
           Conv2d-12            [-1, 1, 26, 26]             577
           Conv2d-13           [-1, 32, 30, 30]             320
  ConvTranspose2d-14            [-1, 1,